In [1]:
## Requirements


Before using this tutorial, ensure that the following are on your system:

- <b>SteganoGAN is installed</b>. Install via pip or source code. 
- <b>Training and Validation Dataset are available </b>. Download via data/download.sh or retrieve your own.

It is also suggested that you have the following:

- <b>CUDA-enabled machine</b>. SteganoGAN takes very long to train without a GPU. Use AWS to have access to CUDA machines.


Now, we retrieve each of the imports required by steganoGAN

## Imports


In [2]:
import numpy as np #numpy is used for a parameter input

In [3]:
from steganogan import SteganoGAN

This imports the SteganoGAN class which has the wrapper functions for SteganoGAN usage:

- <b>Create a SteganoGAN architecture</b> 
- <b>Train a SteganoGAN architecture</b>
- <b>Load a SteganoGAN model</b>
- <b>Encode and decode operations for SteganoGAN models</b>

We retrieve each of these functions later in the tutorial. 

The DataLoader is used to do the following:

- <b>Load images</b> from a selected database
- <b>Specify hyperparameters</b> for database loading


In [4]:
from steganogan.loader import DataLoader

The encoders are the architectural models that are used to encode the messages inside the image. There are two types of encoders that can be imported:

- <b>Basic Encoder</b>: This is memory-efficient but not as robust as the other model
- <b>Dense Encoder</b>: This is a more robust model with a denser architecture

Please review the SteganoGAN paper for images of the two architectures. A steganoGAN model can only use one of these encoders. You may select which one to use in your model. 


In [5]:
from steganogan.encoders import BasicEncoder, DenseEncoder

The decoders are the architectural models that are used to decode the messages inside the image. There are two types of decoders that can be imported:

- <b>Basic Decoder</b>: This is memory-efficient but not as robust as the other model
- <b>Dense Decoder</b>: This is a more robust model with a denser architecture

Please review the SteganoGAN paper for images of the two architectures. A steganoGAN model can only use one of these dector. You may select which one to use in your model. 


In [6]:
from steganogan.decoders import BasicDecoder, DenseDecoder

The Critic checks if an image is steganographic or not. At the current moment, we have the following Critic:

- <b>Basic Critic</b>: This is a GAN discriminator that ensures images are well hid. 

SteganoGAN currently only uses a BasicCritic. This parameter will never be changed 


In [7]:
from steganogan.critics import BasicCritic

## Loading Data


In the next cell, we load in the data for our training and validation dataset. The training dataset is used to train the model while the validation dataset is used to ensure that the model is working correctly. There are several parameters that can you choose to tune.

- <b>path:str</b> - This can be a relative path or an absolute path from the notebook file. 

- <b>limit:int</b> - The number of images you wish to use. If limit is set as np.inf, all the images in the directory will be used.

- <b>shuffle:bool</b> - If true, your images will be randomly shuffled before being used for training.

- <b>batch_size:int</b> - The number of images to use in a batch. A batch represents the number of images that are trained in a single training cycle (i.e. batch_size=10, means 10 images are sent through the network at once during training)

In [8]:
# Load the data
np.inf=1000
train = DataLoader('/data1/guoshi/PaperCode/SteganoGAN/SteganoGANcode/SteganoGAN-master/research/div2k/train/', limit=np.inf, shuffle=True, batch_size=4)
validation = DataLoader('/data1/guoshi/PaperCode/SteganoGAN/SteganoGANcode/SteganoGAN-master/research/div2k/val/', limit=np.inf, shuffle=True, batch_size=4)

## Selecting an Architecture

Below we are deciding on the architecture that we want to use for SteganoGAN. There are several parameters that you can tune here to decide on the architecture. Let us go over them:

- <b>data_depth:int</b> - Represents how many layers we want to represent the data with. Currently, data is representated as a N x data_depth x H x W. Usually, we set this to 1 since that suffices for our needs. For more robustness set this data depth to a higher number.
- <b>encoder:EncoderInstance</b> - You can choose either a BasicEncoder or DenseEncoder.
- <b>decoder:DecoderInstance</b> - You can choose either a DenseEncoder or DenseDecoder.
- <b>critic:CritcInstance</b> - The only option is the BasicCritic
- <b>hidden_size:int</b> - The number of channels we wish to use in the hidden layers of our architecture. You can tune this parameter. We chose 32 as we find relatively good models with these number of channels.
- <b>cuda:bool</b> - If true and the machine is CUDA-enabled, CUDA will be used for training/execution
- <b>verbose:bool</b> - If true, the system will print more output to console

In [9]:
# Create the SteganoGAN instance
steganogan = SteganoGAN(1, BasicEncoder, BasicDecoder, BasicCritic, hidden_size=32, cuda=True, verbose=True)

Using CUDA device


## Training and Saving the Model


Once the architecture has been decided and the training and validation data are we loaded, we can begin training. To train call the fit function with the following parameter options:

- <b>train:DataLoaderInstance</b> - This is the training set that you loaded earlier.
- <b>validation:DataLoaderInstance</b> - This is the validation that you loaded earlier.
- <b>epochs:int</b> - This is the number of epochs you wish to train for. A larger number of epochs will lead to a more precise model. 


In [10]:
# Fit on the given data
steganogan.fit(train, validation, epochs=5)

Epoch 1/5


100%|████████████████████████████████████████████████████████████████████████████████| 25/25 [00:05<00:00,  4.68it/s]


Epoch 2/5


100%|████████████████████████████████████████████████████████████████████████████████| 25/25 [00:05<00:00,  4.52it/s]


Epoch 3/5


100%|████████████████████████████████████████████████████████████████████████████████| 25/25 [00:05<00:00,  4.53it/s]


Epoch 4/5


100%|████████████████████████████████████████████████████████████████████████████████| 25/25 [00:05<00:00,  4.60it/s]


Epoch 5/5


100%|████████████████████████████████████████████████████████████████████████████████| 25/25 [00:05<00:00,  4.60it/s]


Once the model is trained, we save the model to a .steg file. In this file, we save all the model weights and the architectures that these weights compose. Both the encoder and decoder are saved in the same file.

The arguments taken are:
- <b>path:str</b> - This is the path to save the model. Make sure that the directory exists. 

In [11]:
# Save the fitted model
steganogan.save('demo.steg')

## Loading and Executing a Model

The next command loads a previously generated model. It takes a couple of different parameters. 

- <b>architecture:str</b> - You can select either 'basic' or 'dense' architectures. 
- <b>path:str</b> - The path to a model that you have previously generated. 
- <b>cuda:bool</b> - If true and the machine is CUDA-enabled, CUDA will be used for training/execution
- <b>verbose:bool</b> - If true, the system will print more output to console

Note: <b>either architectures or path but not both must not be None</b>

In [15]:
# Load the model
steganogan = SteganoGAN.load(architecture='demo', path=None, cuda=True, verbose=True)

FileNotFoundError: [Errno 2] No such file or directory: '/data1/guoshi/anaconda3/envs/guo_gan/lib/python3.8/site-packages/steganogan/pretrained/demo1.steg'

This function encodes an input image with a message and outputs a steganographic image. Note that since SteganoGAN only works on spatial-domains, the output image must be a PNG image.  

The function takes the following arguments:
- <b>input_image:str</b>: The path to the input image
- <b>output_image:str</b>: The path to the output image
- <b>secret_message:str</b>: The secret message you wish to embed.


In [13]:
# Encode a message in input.png
steganogan.encode('input.png', 'output.png', 'secret and magic')

Encoding completed.


This function decode a steganographic image with a message and outputs a message. If no message is found, an error will be thrown by the function. Since steganoGAN encoders and decoders come in pairs, you <b>must</b> use the decoder that was trained with its corresponding encoder. 

The function takes the following arguments:
- <b>stego_image:str</b>: The path to the steganographic image 


In [14]:
# Decode the message from output.png
steganogan.decode('output.png')

ValueError: Failed to find message.